In [1]:
#Jack Baxter
#Assignment 2.1: Data Lake
#AAI-540

In [2]:
#dependencies, s3 bucket
%store

Stored variables and their in-db values:
s3_assignment_path                    -> 's3://sagemaker-us-east-1-883727203134/'
s3_assignment_path_data               -> 's3://sagemaker-us-east-1-883727203134/dataset.csv
s3_public_path                        -> 's3://sagemaker-us-east-1-883727203134/'
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


In [3]:
#load set to s3

In [4]:
!aws s3 cp dataset.csv s3://sagemaker-us-east-1-883727203134/dataset.csv

upload: ./dataset.csv to s3://sagemaker-us-east-1-883727203134/dataset.csv


In [5]:
!aws s3 ls s3://sagemaker-us-east-1-883727203134

2026-01-20 02:35:49   20118244 dataset.csv


In [6]:
s3_data_path = 's3://sagemaker-us-east-1-883727203134/dataset.csv'
%store s3_data_path
%store

Stored 's3_data_path' (str)
Stored variables and their in-db values:
s3_assignment_path                    -> 's3://sagemaker-us-east-1-883727203134/'
s3_assignment_path_data               -> 's3://sagemaker-us-east-1-883727203134/dataset.csv
s3_data_path                          -> 's3://sagemaker-us-east-1-883727203134/dataset.csv
s3_public_path                        -> 's3://sagemaker-us-east-1-883727203134/'
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


In [7]:
#data lake/athena

In [10]:
import boto3
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [11]:
ingest_create_athena_db_passed = False

In [12]:
#params
database_name = "dsoaws"
table_name_csv = "datasetcsv"

In [14]:
from pyathena import connect
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [15]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [16]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS dsoaws


In [17]:
import pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_326/1322682284.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [18]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_326/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [21]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True
%store ingest_create_athena_db_passed
%store

Stored 'ingest_create_athena_db_passed' (bool)
Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_assignment_path                         -> 's3://sagemaker-us-east-1-883727203134/'
s3_assignment_path_data                    -> 's3://sagemaker-us-east-1-883727203134/dataset.csv
s3_data_path                               -> 's3://sagemaker-us-east-1-883727203134/dataset.csv
s3_public_path                             -> 's3://sagemaker-us-east-1-883727203134/'
setup_dependencies_passed                  -> True
setup_s3_bucket_passed                     -> True


In [22]:
#register csv with athena
import boto3
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [23]:
df = pd.read_csv('dataset.csv')
print(df.dtypes)

Unnamed: 0            int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object


In [ ]:
statement = f"""CREATE EXTERNAL TABLE IF NOT EXISTS
{database_name}.{table_name_csv} (
    track_id            string,
    artists             string,
    album_name          string,
    track_name          string,
    popularity          int,
    duration_ms         int,
    explicit            boolean,
    danceability        float,
    energy              float,
    key                 int,
    loudness            float,
    mode                int,
    speechiness         float,
    acousticness        float,
    instrumentalness    float,
    liveness            float,
    valence             float,
    tempo               float,
    time_signature      int,
    track_genre         string
)
ROW FORMAT DELIMITED 
    FIELDS TERMINATED BY ','
    LINES TERMINATED BY '\\n'
LOCATION '{s3_data_path}'
TBLPROPERTIES ('skip.header.line.count'='1')"""